<a href="https://colab.research.google.com/github/shivam-kotwalia/data-science-project/blob/master/Image_Segmentation_on_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Segmetation on Oxford Pets Dataset 

In [0]:
# Download the dataset 
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
!tar -xvf images.tar.gz
!tar -xvf annotations.tar.gz

In [11]:
import os
import numpy as np 
import pandas as pd
# print(os.getcwd())
print(len(os.listdir("images")))
images_name = os.listdir("images")
images_name = [i.split("_")[0] for i in images_name]
print(images_name[0])
print(pd.Series(images_name).value_counts())

/content
7393
havanese
english          400
american         400
Abyssinian       203
boxer            200
shiba            200
german           200
Sphynx           200
keeshond         200
Siamese          200
wheaten          200
pug              200
great            200
havanese         200
japanese         200
Birman           200
yorkshire        200
basset           200
leonberger       200
samoyed          200
Maine            200
saint            200
chihuahua        200
newfoundland     200
Egyptian         200
Russian          200
British          200
Persian          200
Ragdoll          200
beagle           200
miniature        200
pomeranian       200
Bombay           200
Bengal           200
scottish         199
staffordshire    191
dtype: int64
